In [1]:
import random
import csv
import pandas as pd
import tenseal as ts
import numpy as np
import ast

import fingerprint_feature_extractor
import fingerprint_enhancer
import cv2
import os
import csv

import pickle

In [3]:
# This method was used in early tests to randomly generate data in our desired format
def generate_minutiae(num_fingerprints, num_minutiae_per_fingerprint):
    dataset = []
    for i in range(num_fingerprints):
        fingerprint = []
        fingerprint_id = i+1
        for _ in range(num_minutiae_per_fingerprint):
            x = random.randint(0, 500) # x coord
            y = random.randint(0, 500) # y coord
            angle = random.randint(0, 360) # angle of minutiae
            minutiae_type = random.randint(0, 1) #0 for ridge, 1 for bif
            fingerprint.append((fingerprint_id, x, y, angle, minutiae_type)) # add to individual fingerprint data
        dataset.append(fingerprint)
    return dataset

# Saves minutiae points to .csv file for ease of use later
def save_to_csv(dataset, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ID', 'X', 'Y', 'Orientation', 'Type'])
        for fingerprint in dataset:
            for minutiae in fingerprint:
                writer.writerow(minutiae)

# UNUSED - Encrypts a dataframe as a dictionary, with ID maintained but minutiae encrypted
def encrypt_fingerprint_data(df, context):
    encrypted_minutiae_dict = {}
    for fingerprint_id in df['ID'].unique():
        fingerprint_minutiae = df[df['ID'] == fingerprint_id][['X', 'Y', 'Orientation', 'Type']].values.flatten().tolist()
        scaled_minutiae = [float(value) for value in fingerprint_minutiae]
        encrypted_minutiae = ts.ckks_vector(context, scaled_minutiae)
        encrypted_minutiae_dict[fingerprint_id]=encrypted_minutiae
    return encrypted_minutiae_dict

# Unused concept of finding a specific match in a database
def find_matching_fingerprint_id(encrypted_query, encrypted_data, context):
    min_distance = float('inf')
    matching_id = None

    for fingerprint_id, encrypted_minutiae in encrypted_data.items():
        difference_vector = encrypted_query - encrypted_minutiae
        distance_squared = difference_vector.dot(difference_vector)

        distance_squared_decrypted = distance_squared.decrypt()
        distance = np.sqrt(distance_squared_decrypted)

        if distance < min_distance:
            min_distance = distance
            matching_id = fingerprint_id

    return matching_id

# Unused method for normalizing values
def normalize_and_scale_minutiae(df):
    df['locX'] = (df['locX'] / df['locX'].max())
    df['locY'] = (df['locY'] / df['locY'].max())
    #df['Orientation'] = df['Orientation'] / 360
    # No change for 'Type' as it's categorical (0 or 1)
    return df

# Used to pad the dataframe for compatibility with tensor operations
def pad_or_truncate_minutiae_list(minutiaeList, targetLength):
    # Define dummy minutiae with impossible coordinates and orientation as placeholder
    dummy_minutiae = fingerprint_feature_extractor.MinutiaeFeature(-1, -1, float('nan'), 'Dummy')
    currentLength = len(minutiaeList)
    
    if currentLength > targetLength:
        return minutiaeList[:targetLength]
    else:
        return minutiaeList + [dummy_minutiae] * (targetLength - currentLength)

# Used for preprocessing a folder of fingerprint images
def process_images_from_folder(folder_path, csv_filepath, target_minutiae_count, enrollment = True):
    headers = ['ID', 'Type', 'LocationX', 'LocationY', 'Orientation']
    
    with open(csv_filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        ID = 0
        for filename in os.listdir(folder_path):
            if filename.endswith(".tif"):  # Check for TIFF images, adjust if using other formats
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path, 0)
                out = fingerprint_enhancer.enhance_Fingerprint(img)
                
                # Process image to enhance and extract features, replace with your actual method
                # out = fingerprint_enhancer.enhance_Fingerprint(img)
                if(enrollment==False):
                    cv2.imshow('enhanced_image', out);
                    cv2.waitKey(0)
                    FeaturesTerminations, FeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(
                                    out, spuriousMinutiaeThresh=10, invertImage=False, showResult=True, saveResult=True)
                else:
                    FeaturesTerminations, FeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(
                                    out, spuriousMinutiaeThresh=10, invertImage=False, showResult=False, saveResult=True)
                
                # Ensure a fixed number of minutiae
                FeaturesTerminations = pad_or_truncate_minutiae_list(FeaturesTerminations, target_minutiae_count//2)
                FeaturesBifurcations = pad_or_truncate_minutiae_list(FeaturesBifurcations, target_minutiae_count//2)

                # Write termination features
                for feature in FeaturesTerminations:
                    writer.writerow([ID, 0, feature.locX, feature.locY, feature.Orientation])
                
                # Write bifurcation features
                for feature in FeaturesBifurcations:
                    writer.writerow([ID, 1, feature.locX, feature.locY, feature.Orientation])
            ID+=1

# Used to encrypt the whole dataframe to dictionary form
# ID: (encrypted data)
def encryptPrintsDatabase(df, context):
    total_prints = len(pd.unique(df['ID']))
    database = {}
    for i in range(0, total_prints):
        fingerprint_minutiae = df[df['ID'] == i][['LocationX', 'LocationY', 'Orientation', 'Type']].values.flatten().tolist()
        scaled_minutiae = []

        for value in fingerprint_minutiae:
            try:
                # Attempt to convert directly to float
                converted_value = float(value)
                if np.isnan(converted_value):
                    converted_value = 0  # Replace NaN with 0 or another appropriate value
                scaled_minutiae.append(converted_value)
            except ValueError:
                # If direct conversion fails, try evaluating as a list and then convert each element
                try:
                    list_values = ast.literal_eval(value)
                    if isinstance(list_values, list):
                        scaled_list_values = [float(v) if not np.isnan(float(v)) else 0 for v in list_values]  # Replace NaN within the list
                        scaled_minutiae.extend(scaled_list_values)
                    else:
                        raise ValueError
                except ValueError:
                    # If conversion still fails, handle or report the erroneous value
                    print(f"Could not convert value: {value}")

        val = 0
        target_length = 1000

        while len(scaled_minutiae) < target_length:
            scaled_minutiae.append(val)
        arr = np.array(scaled_minutiae)
        enc = ts.ckks_vector(context, arr)
        database[i] = enc
    
    return database

In [4]:
def process_image_to_dataframe(ID, img_path, file_path, target_minutiae_count):
    headers = ['ID', 'Type', 'LocationX', 'LocationY', 'Orientation']
    
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)

        img = cv2.imread(img_path, 0)
        out = fingerprint_enhancer.enhance_Fingerprint(img)
        cv2.imshow('enhanced_image', out);
        cv2.waitKey(0)
        # Extract features
        FeaturesTerminations, FeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(
                        out, spuriousMinutiaeThresh=10, invertImage=False, showResult=True, saveResult=True)

        FeaturesTerminations = pad_or_truncate_minutiae_list(FeaturesTerminations, target_minutiae_count//2)
        FeaturesBifurcations = pad_or_truncate_minutiae_list(FeaturesBifurcations, target_minutiae_count//2)

    # Write termination features
        for feature in FeaturesTerminations:
            writer.writerow([ID, 0, feature.locX, feature.locY, feature.Orientation])
        
        # Write bifurcation features
        for feature in FeaturesBifurcations:
            writer.writerow([ID, 1, feature.locX, feature.locY, feature.Orientation])
    df = pd.read_csv(file_path)
    return df

def encryptSinglePrint(df, ID, context):
    fingerprint_minutiae = df[df['ID'] == ID][['LocationX', 'LocationY', 'Orientation', 'Type']].values.flatten().tolist()
    scaled_minutiae = []
    for value in fingerprint_minutiae:
            try:
                # Attempt to convert directly to float
                converted_value = float(value)
                if np.isnan(converted_value):
                    converted_value = 0  # Replace NaN with 0 or another appropriate value
                scaled_minutiae.append(converted_value)
            except ValueError:
                # If direct conversion fails, try evaluating as a list and then convert each element
                try:
                    list_values = ast.literal_eval(value)
                    if isinstance(list_values, list):
                        scaled_list_values = [float(v) if not np.isnan(float(v)) else 0 for v in list_values]  # Replace NaN within the list
                        scaled_minutiae.extend(scaled_list_values)
                    else:
                        raise ValueError
                except ValueError:
                    # If conversion still fails, handle or report the erroneous value
                    print(f"Could not convert value: {value}")

    val = 0
    target_length = 1000

    while len(scaled_minutiae) < target_length:
        scaled_minutiae.append(val)
    arr = np.array(scaled_minutiae)
    encrypted_vec = ts.ckks_vector(context, arr)
    return encrypted_vec

#### Enrollment (entire database)

In [5]:
# For Demo Purposes - these statements define the folder we will
# enroll for storage in our 'database'

folder_path = r"C:\Users\quinn\Desktop\LiveDemo\EnrollmentSet"
csv_filepath = r"C:\Users\quinn\Desktop\LiveDemo\database.csv"
target_minutiae_count = 200

process_images_from_folder(folder_path, csv_filepath, target_minutiae_count)

In [6]:
df = pd.read_csv(r"C:\Users\quinn\Desktop\LiveDemo\database.csv")
df

print(df)

      ID  Type  LocationX  LocationY          Orientation
0      0     0         85        238             [-180.0]
1      0     0         87        273               [-0.0]
2      0     0         96        255               [-0.0]
3      0     0        103        293   [153.434948822922]
4      0     0        112        196  [-153.434948822922]
...   ..   ...        ...        ...                  ...
1395   6     1         -1         -1                  NaN
1396   6     1         -1         -1                  NaN
1397   6     1         -1         -1                  NaN
1398   6     1         -1         -1                  NaN
1399   6     1         -1         -1                  NaN

[1400 rows x 5 columns]


In [7]:
# Generating TenSEAL context

# - Contains public key, private key, and homomorphic context
#      for doing computations (add, mult, matrix)
#
# - Defines the scheme (CKKS), the degree to which the polynomial
#      can calculate floating point values, the bit sizes of 
#      coefficients values in the polynomial, and the encryption type
#      
# - poly_modulus_degree affects the following:
#       1) Number of coefficients in plaintext polynomials
#       2) Size of ciphertext elements
#       3) Computational performance (big=bad!)
#       4) Security level (big=good!)
#
# - coeff_mod_bit_sizes refers to a list of primes (coefficient modulus)
#       which affects the following:
#       1) Size of ciphertext elements
#       2) Number of encrypted multiplications supported
#       3) The security level (big=bad)
#       Note: Each of the prime numbers in the coeff. modulus must be
#             at most 60bits and congruent to 1 mod 2*poly_modulus_degree

context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=16384,
    coeff_mod_bit_sizes=[60, 40, 40, 40, 60],
    encryption_type=ts.ENCRYPTION_TYPE.ASYMMETRIC
)

context.global_scale = 2**40
context.generate_galois_keys()
context.generate_relin_keys()
secret_key = context.secret_key()

In [8]:
# This statement is encrypting every entry in the dataframe
# and saving it as a dictionary, with ID: Data as the format

encrypted_database = encryptPrintsDatabase(df, context)

In [9]:
# The following is a simple method to save the database for 
# server storage or transfer. Other more sophisticated
# methods would be desired for actual cloud storage and 
# computations.

pickle_dict = {}
for i in range(0, 7):
    buff = encrypted_database[i].serialize()
    pickle_dict[i] = buff
with open('saved_database.pkl', 'wb') as f:
    pickle.dump(pickle_dict, f)

In [10]:
# Similar to the last block, these statements show how the 
# public key and private key can be saved to securely
# transfer to a trusted party for decryption of 
# computation results.


public_key_pickle = context.serialize(save_secret_key=False)
private_key_pickle = context.serialize(save_secret_key=True)

with open('public_key.key', 'wb') as key_file:
    key_file.write(public_key_pickle)

with open('private_key.key', 'wb') as pkey_file:
    pkey_file.write(private_key_pickle)

In [11]:
# These statements show the database and PUBLIC key
# being loaded from the previously saved file, and the
# database being 'recontextualized' after file transfer

with open('saved_database.pkl', 'rb') as f:
    loaded_pickle_dict = pickle.load(f)

with open('public_key.key', 'rb') as k:
    public_context_dat = k.read()
    public_context = ts.context_from(public_context_dat)
    
new_vectors = {}
for i, serialized_data in loaded_pickle_dict.items():
    vector = ts.lazy_ckks_vector_from(serialized_data)
    vector.link_context(public_context)
    new_vectors[i] = vector

In [12]:
# Performing homomorphic encrypted calculations
# Two example: subtraction and multiplication, yields encrypted vectors
# If decrypted, they show the corresponding values
sub = new_vectors[1] - new_vectors[2]
mult = new_vectors[1]*new_vectors[2]
print(sub)
print(mult)

print(sub.decrypt(secret_context.secret_key()))
print(mult.decrypt(secret_context.secret_key()))


NameError: name 'secret_context' is not defined

In [13]:
# This will be the image we will use to verify matching data

img_path = r"C:\Users\quinn\Desktop\LiveDemo\AuthenticationSet\76\076_3_5.tif"
client_print = process_image_to_dataframe(1, img_path, "client.csv", 200)
client_print

encrypted_client = encryptSinglePrint(client_print, 1, public_context)

# These lines allow for saving and transferring of encrypted data
buff = encrypted_client.serialize()
request = {"1":buff}
with open('saved_client.pkl', 'wb') as f:
    pickle.dump(request, f)

In [38]:
# Loading encrypted client data
with open('saved_client.pkl', 'rb') as r:
    loaded_client = pickle.load(r)

# A process called recontextualizing, after storage
client_data = {}
for i, serialized_data in loaded_client.items():
    vector = ts.lazy_ckks_vector_from(serialized_data)
    vector.link_context(public_context)
    client_data[i] = vector

# Loading the private context/key from storage
with open('private_key.key', 'rb') as pk:
    secret_context_dat = pk.read()
    secret_context = ts.context_from(secret_context_dat)

# Distance between minutiae points of the saved data and query data
distance_squared = (new_vectors[6] - client_data['1']).dot(new_vectors[6] - client_data['1'])
verify_dist = distance_squared.decrypt(secret_context.secret_key())

print("Distance between points in successful verification:", verify_dist)

distance_squared2 = (new_vectors[5] - client_data['1']).dot(new_vectors[5] - client_data['1'])
verify_dist2 = distance_squared2.decrypt(secret_context.secret_key())

print("Distance between points in unsuccessful verification:", verify_dist2)

Distance between points in successful verification: [1.749625275640262e+30]
Distance between points in unsuccessful verification: [5.117389056009587e+31]
